# RunPod ComfyUI Initialization (Modular Approach)

This notebook demonstrates the new modular, functional approach to RunPod deployments. It uses the updated configuration system for managing favorite images and GPU configurations.

## Prerequisites
- RunPod API key set in `.env` file
- `runpod` Python package installed
- Optional: RunPod Network Volume ID for persistent storage

In [ ]:
# Install required packages if not already installed
!pip install runpod python-dotenv ipywidgets tqdm requests

In [ ]:
import sys
import os
import time
import json
import requests

from dotenv import load_dotenv
from datetime import datetime
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Add src to path so we can import our modules
sys.path.append('../../src')

from infra.runpod_manager import RunPodManager
from infra.runpod_config import (
    list_favorite_images,
    list_gpu_configs,
    get_image_details,
    get_gpu_details,
    get_deployment_config,
    recommend_deployment_config
)

## Configuration and Authentication

In [ ]:
# Load environment variables
load_dotenv()

RUNPOD_API_KEY = os.getenv("RUNPOD_API_KEY")
if not RUNPOD_API_KEY:
    raise ValueError("RUNPOD_API_KEY environment variable not set. Please set it in your .env file.")

print(f"RunPod API key loaded: {RUNPOD_API_KEY[:8]}...{RUNPOD_API_KEY[-4:]}")

## Available Deployment Options

Let's explore the new modular configuration system that tracks favorite images and GPU configurations.

In [ ]:
def display_available_options():
    """Display all available images and GPU configurations."""
    print("=== Available Favorite Images ===")
    for image_key in list_favorite_images():
        details = get_image_details(image_key)
        print(f"  {image_key}: {details['description']}")
        print(f"    Default GPU: {details['default_gpu']}")
        print(f"    Tags: {', '.join(details['tags'])}")
        print()

    print("=== Available GPU Configurations ===")
    for gpu_key in list_gpu_configs():
        details = get_gpu_details(gpu_key)
        print(f"  {gpu_key}: {details['name']} ({details['vram_gb']}GB VRAM)")
        print(f"    Cost effectiveness: {details['cost_effectiveness']}")
        print(f"    Use cases: {', '.join(details['use_cases'])}")
        print()

display_available_options()

## Initialize RunPod Manager

Create an instance of the RunPodManager to handle our deployments.

In [ ]:
# Initialize RunPod manager
try:
    manager = RunPodManager(RUNPOD_API_KEY)
    print("✅ RunPod manager initialized successfully!")
except Exception as e:
    print(f"❌ Failed to initialize RunPod manager: {e}")
    manager = None

## Deployment Examples

Let's demonstrate the various deployment approaches with the new modular system.

In [ ]:
def deploy_pod_example_1():
    """Deploy using a preset configuration - the simplest approach."""
    if not manager:
        print("Manager not initialized")
        return None
        
    print("🚀 Deploying ComfyUI with default configuration...")
    try:
        pod = manager.create_pod_from_config(
            name="comfyui-modular-default",
            config_name="comfyui-default"
        )
        print(f"✅ Pod created successfully with ID: {pod['id']}")
        return pod
    except Exception as e:
        print(f"❌ Failed to create pod: {e}")
        return None

def deploy_pod_example_2():
    """Deploy optimized for a specific use case with budget constraints."""
    if not manager:
        print("Manager not initialized")
        return None
        
    print("🚀 Deploying ComfyUI optimized for large models with low budget...")
    try:
        pod = manager.create_pod_for_use_case(
            name="comfyui-budget-optimized",
            use_case="comfyui-large",
            budget_level="low",
            required_vram_gb=20
        )
        print(f"✅ Pod created successfully with ID: {pod['id']}")
        return pod
    except Exception as e:
        print(f"❌ Failed to create pod: {e}")
        return None

def deploy_pod_example_3():
    """Deploy with custom overrides to a preset configuration."""
    if not manager:
        print("Manager not initialized")
        return None
        
    print("🚀 Deploying Jupyter ML environment with custom volume size...")
    try:
        pod = manager.create_pod_from_config(
            name="jupyter-ml-custom",
            config_name="jupyter-ml",
            volume_in_gb=200,  # Override default volume size
            env={
                "JUPYTER_ENABLE_LAB": "yes",
                "CUSTOM_ENV_VAR": "custom_value"
            }
        )
        print(f"✅ Pod created successfully with ID: {pod['id']}")
        return pod
    except Exception as e:
        print(f"❌ Failed to create pod: {e}")
        return None

# Run the examples
pod1 = deploy_pod_example_1()
pod2 = deploy_pod_example_2()
pod3 = deploy_pod_example_3()

## Wait for Pods to be Ready

Let's wait for our deployed pods to become ready for use.

In [ ]:
def wait_for_pods_ready(pods, timeout=300):
    """Wait for multiple pods to become ready."""
    if not manager:
        print("Manager not initialized")
        return
        
    ready_pods = []
    for pod in pods:
        if not pod:
            continue
            
        pod_id = pod['id']
        print(f"⏳ Waiting for pod {pod_id} to become ready (timeout: {timeout}s)..." )
        ready_pod = manager.wait_for_pod_running(pod_id, timeout)
        if ready_pod:
            print(f"✅ Pod {pod_id} is ready!")
            ready_pods.append(ready_pod)
        else:
            print(f"❌ Timed out waiting for pod {pod_id} to become ready")
    
    return ready_pods

# Wait for our deployed pods
deployed_pods = [pod for pod in [pod1, pod2, pod3] if pod is not None]
ready_pods = wait_for_pods_ready(deployed_pods)

## List Active Pods

Let's check what pods we currently have running.

In [ ]:
def list_active_pods():
    """List all active pods."""
    if not manager:
        print("Manager not initialized")
        return
        
    try:
        pods = manager.get_active_pods()
        print(f"\n📋 Found {len(pods)} active pod(s):\n")
        
        for pod in pods:
            print(f"  Name: {pod.get('name', 'N/A')}")
            print(f"  ID: {pod.get('id', 'N/A')}")
            print(f"  Status: {pod.get('desiredStatus', 'N/A')}")
            print(f"  GPU: {pod.get('gpuTypeId', 'N/A')}")
            print(f"  Created: {pod.get('createdAt', 'N/A')}")
            print("  " + "-" * 40)
    except Exception as e:
        print(f"❌ Failed to list pods: {e}")

list_active_pods()

## Clean Up (Optional)

If you want to clean up the pods created in this notebook, uncomment and run the following cell.

In [ ]:
# ⚠️ WARNING: This will terminate all active pods!
# Only run this if you want to clean up all pods created in this session.

# def cleanup_session_pods():
#     """Clean up pods created in this session."""
#     session_pods = ["comfyui-modular-default", "comfyui-budget-optimized", "jupyter-ml-custom"]
#     
#     if not manager:
#         print("Manager not initialized")
#         return
#         
#     try:
#         all_pods = manager.get_active_pods()
#         terminated_count = 0
#         
#         for pod in all_pods:
#             if pod.get('name') in session_pods:
#                 pod_id = pod.get('id')
#                 print(f"Terminating pod: {pod.get('name')} (ID: {pod_id})")
#                 try:
#                     manager.terminate_pod(pod_id)
#                     terminated_count += 1
#                 except Exception as e:
#                     print(f"  - Error terminating pod {pod_id}: {e}")
#         
#         print(f"\n✅ Terminated {terminated_count} session pods")
#     except Exception as e:
#         print(f"❌ Failed to clean up pods: {e}")

# # Uncomment the line below to run cleanup
# # cleanup_session_pods()

## Summary

This notebook demonstrated the new modular, functional approach to RunPod deployments:

1. **Configuration System**: We now have a centralized configuration system that tracks favorite images and GPU configurations.

2. **Deployment Methods**:
   - `create_pod_from_config()`: Deploy using predefined configurations
   - `create_pod_for_use_case()`: Deploy optimized for specific use cases with budget constraints
   - `create_pod()`: Traditional custom deployment (still available)

3. **GPU Selection Logic**: The system automatically recommends appropriate GPUs based on:
   - VRAM requirements
   - Budget constraints
   - Use case requirements

4. **Common GPU Configurations**:
   - **96GB VRAM**: Used when no budget is set and maximum resources are needed
   - **5090**: Sometimes used for large models ("-large" tag)
   - **3090**: Cost-effective default option

This approach makes deployments more consistent, reproducible, and easier to manage across different environments and use cases.